In [1]:
import numpy as np
import math
import mesa
from mesa.experimental import JupyterViz
import numpy as np

In [19]:
class Voter(mesa.Agent):
    """
    An agent representing the behavior of a voter.

    uid: the unique ID of the voter
    opinion: the array representing the voter's opinions as well as their position
    """

    def __init__(self, uid, model):
        super().__init__(uid, model)
        self.unique_id = uid
        self.opinion = np.zeros(2)


    def step(self):
        """
        TODO: Add logic for voter to cast their vote to the nearest candidate
        TODO: Determine how a voter moves between elections
        """
        candidate = self.get_nearest_candidate()
        if candidate:
            candidate.num_votes += 1


    def get_nearest_candidate(self):
        """
        Returns the nearest candidate to the Voter.
        """
        candidates = self.model.agents[Candidate]
        min_candidate = None
        min_dist = math.inf
        for candidate in candidates:
            dist = np.linalg.norm(self.opinion - candidate.opinion)
            if dist < min_dist:
                min_dist = dist
                min_candidate = candidate

        return min_candidate



class Candidate(mesa.Agent):
    """
    An agent representing the behavior of a cadndiate.

    uid: the unique ID of the candidate
    opinion: the array representing the candidate's opinions as well as their position
    num_votes: the number of votes that candidate recieved
    """


    def __init__(self, uid, model):
        super().__init__(uid, model)
        self.unique_id = uid
        self.opinion = np.zeros(2)
        self.num_votes = 0


    def step(self):
        """
        TODO: Determine how a candidate moves between elections
        """
        pass


class ElectionSystem(mesa.Model):
    """
    The model representing the plurality election system.

    num_agents: total number of agents
    num_voters: number of voting agents
    num_candidates: number of candidate agents
    winner: the winning candidate of the election
    agents: all agents in model stored as a dict of {"Agent Type":[agent1, agent2, ..., agentN]}
    """


    def __init__(self, num_voters, num_candidates, width, height):
        super().__init__()
        self.num_agents = num_voters + num_candidates
        self.num_voters = num_voters
        self.num_candidates = num_candidates
        self.winner = None
        self.agents = {}
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.MultiGrid(width=width, height=height, torus=True)

        # initialize voters
        self.agents[Voter] = []
        for i in range(self.num_voters):
            voter = Voter(i, self)
            self.schedule.add(voter)
            self.agents[Voter].append(voter)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(voter, (x, y))

            # voter opinion is dictated by their location
            voter.opinion = np.array([x, y])

        # initialize candidates
        self.agents[Candidate] = []
        for i in range(self.num_voters, self.num_agents):
            candidate = Candidate(i, self)
            self.schedule.add(candidate)
            self.agents[Candidate].append(candidate)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(candidate, (x, y))

            # voter opinion is dictated by their location
            candidate.opinion = np.array([x, y])

        # data collector
        self.datacollector = mesa.datacollection.DataCollector(
            model_reporters = {"winner": "winner"}
        )

        self.running = True
        self.datacollector.collect(self)


    def step(self):
        """
        TODO: Count each candidates votes to determine winner.
        """
        
        # count votes to determine winner
        candidates = self.agents[Candidate]
        most_votes = 0
        for candidate in candidates:
            if candidate.num_votes > most_votes:
                self.winner = candidate.unique_id
                most_votes = candidate.num_votes

        if self.winner: print(self.winner)

        self.datacollector.collect(self)
        self.schedule.step()

In [20]:
model = ElectionSystem(7, 2, 10, 10)
for i in range(5):
    model.step()

8
8
8
8


In [23]:
def agent_portrayal(agent):
    if isinstance(agent, Voter):
        return {
            "color": "tab:blue",
            "size": 50,
        }
    elif isinstance(agent, Candidate):
        return {
            "color": "tab:red",
            "size": 100,
        }


model_params = {
    "num_voters": {
        "type": "SliderInt",
        "value": 7,
        "label": "Number of voters:",
        "min": 3,
        "max": 25,
        "step": 1,
    },
    "num_candidates": 2,
    "width": 10,
    "height": 10,
}

page = JupyterViz(
    ElectionSystem,
    model_params,
    measures=["winner"],
    name="Plurality Election System",
    agent_portrayal=agent_portrayal,
)
# This is required to render the visualization in the Jupyter notebook
page


Cannot show ipywidgets in text

10
10
10
10
10
10
